In [2]:
import sys
!{sys.executable} -m pip install wefe

In [3]:
from wefe.query import Query
from wefe.word_embedding_model import WordEmbeddingModel
from wefe.metrics.WEAT import WEAT
from wefe.datasets.datasets import load_weat
import gensim.downloader as api
from wefe.utils import run_queries, create_ranking, plot_ranking, plot_ranking_correlations
from gensim.models import KeyedVectors

In [7]:
import gensim
import os
import shutil
import hashlib
from sys import platform

def getFileLineNums(filename):
    f = open(filename, 'r')
    count = 0
    for line in f:
        count += 1
    return count
 
 #Linux or Windows open the word vector file, add a line at the beginning
def prepend_line(infile, outfile, line):
    with open(infile, 'r') as old:
        with open(outfile, 'w') as new:
            new.write(str(line) + "\n")
            shutil.copyfileobj(old, new)
 
def prepend_slow(infile, outfile, line):
    with open(infile, 'r') as fin:
        with open(outfile, 'w') as fout:
            fout.write(line + "\n")
            for line in fin:
                fout.write(line)

def load(filename):
    num_lines = getFileLineNums(filename)
    gensim_file = 'glove.840B.300d duplicate.txt'
    gensim_first_line = "{} {}".format(num_lines, 300)
    # Prepends the line.
    if platform == "linux" or platform == "linux2":
        prepend_line(filename, gensim_file, gensim_first_line)
    else:
        prepend_slow(filename, gensim_file, gensim_first_line)
    model = gensim.models.KeyedVectors.load_word2vec_format(gensim_file)

In [5]:
glove_1 = WordEmbeddingModel(api.load('glove-twitter-25'),
                             'glove twitter dim=25')
glove_2 = WordEmbeddingModel(api.load('glove-twitter-50'),
                             'glove twitter dim=50')
glove_3 = WordEmbeddingModel(api.load('glove-twitter-100'),
                             'glove twitter dim=100')
glove_4 = WordEmbeddingModel(api.load('glove-twitter-200'),
                             'glove twitter dim=200')

INFO:gensim.models.keyedvectors:loading projection weights from /Users/EmpiricalSE/gensim-data/glove-twitter-25/glove-twitter-25.gz
DEBUG:smart_open.smart_open_lib:{'uri': '/Users/EmpiricalSE/gensim-data/glove-twitter-25/glove-twitter-25.gz', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (1193514, 25) matrix of type float32 from /Users/EmpiricalSE/gensim-data/glove-twitter-25/glove-twitter-25.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-08-11T21:17:43.309072', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.14.5-x86_64-i386-64bit', 'event': 'load_word2vec_format'}
INFO:gensim.models.keyedvectors:loading projection weights from /Users/

In [8]:
glove_5 = WordEmbeddingModel(api.load('glove-wiki-gigaword-50'),
                             'glove wiki dim=50')
glove_6 = WordEmbeddingModel(api.load('glove-wiki-gigaword-100'),
                             'glove wiki dim=100')
glove_7 = WordEmbeddingModel(api.load('glove-wiki-gigaword-200'),
                             'glove wiki dim=200')
glove_8 = WordEmbeddingModel(api.load('glove-wiki-gigaword-300'),
                             'glove wiki dim=300')

INFO:gensim.models.keyedvectors:loading projection weights from /Users/EmpiricalSE/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz
DEBUG:smart_open.smart_open_lib:{'uri': '/Users/EmpiricalSE/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (400000, 50) matrix of type float32 from /Users/EmpiricalSE/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-08-11T21:24:08.335209', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.14.5-x86_64-i386-64bit', 'event': 'load_word2vec_format'}
INFO:gensim.models.keyedvectors:load

In [9]:
load('glove.840B.300d.txt')

INFO:gensim.models.keyedvectors:loading projection weights from glove.840B.300d duplicate.txt
DEBUG:smart_open.smart_open_lib:{'uri': 'glove.840B.300d duplicate.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (2196017, 300) matrix of type float32 from glove.840B.300d duplicate.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-08-11T21:33:22.837803', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.14.5-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


In [10]:
glove_embeddings_9 = KeyedVectors.load_word2vec_format('glove.42B.300d duplicate.txt')
glove_9 = WordEmbeddingModel(glove_embeddings_9, 'glove common crawl 42B dim=300')
glove_embeddings_10 = KeyedVectors.load_word2vec_format('glove.840B.300d duplicate.txt')
glove_10 = WordEmbeddingModel(glove_embeddings_10, 'glove common crawl 840B dim=300')

INFO:gensim.models.keyedvectors:loading projection weights from glove.42B.300d duplicate.txt
DEBUG:smart_open.smart_open_lib:{'uri': 'glove.42B.300d duplicate.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (1917494, 300) matrix of type float32 from glove.42B.300d duplicate.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-08-11T21:38:52.518635', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.14.5-x86_64-i386-64bit', 'event': 'load_word2vec_format'}
INFO:gensim.models.keyedvectors:loading projection weights from glove.840B.300d duplicate.txt
DEBUG:smart_open.smart_open_lib:{'uri': 'glove.840B.300d duplicate.txt', 'mode': 'rb', 'buffe

In [14]:
fasttext_1 = WordEmbeddingModel(api.load('fasttext-wiki-news-subwords-300'),
                             'fasttext wiki subwords dim=300')
fast_embeddings_2 = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')
fasttext_2 = WordEmbeddingModel(fast_embeddings_2, 'fasttext wiki dim=300')

INFO:gensim.models.keyedvectors:loading projection weights from /Users/EmpiricalSE/gensim-data/fasttext-wiki-news-subwords-300/fasttext-wiki-news-subwords-300.gz
DEBUG:smart_open.smart_open_lib:{'uri': '/Users/EmpiricalSE/gensim-data/fasttext-wiki-news-subwords-300/fasttext-wiki-news-subwords-300.gz', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (999999, 300) matrix of type float32 from /Users/EmpiricalSE/gensim-data/fasttext-wiki-news-subwords-300/fasttext-wiki-news-subwords-300.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-08-11T22:08:11.282672', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.14.5-x86_64-i386-64bit', 'event': 'load

In [15]:
fast_embeddings_3 = KeyedVectors.load_word2vec_format('crawl-300d-2M.vec')
fasttext_3 = WordEmbeddingModel(fast_embeddings_3, 'fasttext crawl dim=300')
fast_embeddings_4 = KeyedVectors.load_word2vec_format('crawl-300d-2M-subword.vec')
fasttext_4 = WordEmbeddingModel(fast_embeddings_4, 'fasttext crawl subwords dim=300')

INFO:gensim.models.keyedvectors:loading projection weights from crawl-300d-2M.vec
DEBUG:smart_open.smart_open_lib:{'uri': 'crawl-300d-2M.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (1999995, 300) matrix of type float32 from crawl-300d-2M.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-08-11T22:16:39.664306', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.14.5-x86_64-i386-64bit', 'event': 'load_word2vec_format'}
INFO:gensim.models.keyedvectors:loading projection weights from crawl-300d-2M-subword.vec
DEBUG:smart_open.smart_open_lib:{'uri': 'crawl-300d-2M-subword.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': No

In [16]:
model_w2v = WordEmbeddingModel(api.load('word2vec-google-news-300'),
                             'word2vec google dim=300')

INFO:gensim.models.keyedvectors:loading projection weights from /Users/EmpiricalSE/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
DEBUG:smart_open.smart_open_lib:{'uri': '/Users/EmpiricalSE/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from /Users/EmpiricalSE/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2021-08-11T22:23:28.056832', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.14.5-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


# Defining queries based on the original WEAT paper

In [17]:
# Load the wordset
weat_wordset = load_weat()

# Define the 10 Queries:
queries = [
    # Flowers vs Insects wrt Pleasant (5) and Unpleasant (5)
    Query([weat_wordset['flowers'], weat_wordset['insects']],
          [weat_wordset['pleasant_5'], weat_wordset['unpleasant_5']],
          ['Flowers', 'Insects'], ['Pleasant(5)', 'Unpleasant(5)']),

    # Instruments vs Weapons wrt Pleasant (5) and Unpleasant (5)
    Query([weat_wordset['instruments'], weat_wordset['weapons']],
          [weat_wordset['pleasant_5'], weat_wordset['unpleasant_5']],
          ['Instruments', 'Weapons'], ['Pleasant(5)', 'Unpleasant(5)']),

    # European american names(5) vs African american names(5)
    # wrt Pleasant (5) and Unpleasant (5)
    Query([
        weat_wordset['european_american_names_5'],
        weat_wordset['african_american_names_5']
    ], [weat_wordset['pleasant_5'], weat_wordset['unpleasant_5']],
          ['European american names(5)', 'African american names(5)'],
          ['Pleasant(5)', 'Unpleasant(5)']),

    # European american names(7) vs African american names(7)
    # wrt Pleasant (5) and Unpleasant (5)
    Query([
        weat_wordset['european_american_names_7'],
        weat_wordset['african_american_names_7']
    ], [weat_wordset['pleasant_5'], weat_wordset['unpleasant_5']],
          ['European american names(7)', 'African american names(7)'],
          ['Pleasant(5)', 'Unpleasant(5)']),

    # European american names(7) vs African american names(7)
    # wrt Pleasant (9) and Unpleasant (9)
    Query([
        weat_wordset['european_american_names_7'],
        weat_wordset['african_american_names_7']
    ], [weat_wordset['pleasant_9'], weat_wordset['unpleasant_9']],
          ['European american names(7)', 'African american names(7)'],
          ['Pleasant(9)', 'Unpleasant(9)']),

    # Male and female names wrt Career and family
    Query([weat_wordset['male_names'], weat_wordset['female_names']],
          [weat_wordset['career'], weat_wordset['family']],
          ['Male names', 'Female names'], ['Career', 'Family']),

    # Math and arts wrt male and female terms
    Query([weat_wordset['math'], weat_wordset['arts']],
          [weat_wordset['male_terms'], weat_wordset['female_terms']],
          ['Math', 'Arts'], ['Male terms', 'Female terms']),

    # Science and arts wrt male and female terms
    Query([weat_wordset['science'], weat_wordset['arts_2']],
          [weat_wordset['male_terms'], weat_wordset['female_terms']],
          ['Science', 'Arts 2'], ['Male terms', 'Female terms']),

    # Mental and Physical disease wrt Temporary and Permanent
    Query([weat_wordset['mental_disease'], weat_wordset['physical_disease']],
          [weat_wordset['temporary'], weat_wordset['permanent']],
          ['Mental disease', 'Physical disease'], ['Temporary', 'Permanent']),

    # Young people names and Old people names disease wrt Pleasant(9) and Unpleasant(9)
    Query(
        [weat_wordset['young_people_names'], weat_wordset['old_people_names']],
        [weat_wordset['pleasant_9'], weat_wordset['unpleasant_9']],
        ['Young peoples names', 'Old peoples names'],
        ['Pleasant(9)', 'Unpleasant(9)'])
]

# GloVe
1. Wikipedia 2014 + Gigaword 5 (6B tokens, 400K vocab, uncased, 50d, 100d, 200d, & 300d vectors, 822 MB download
2. Common Crawl (42B tokens, 1.9M vocab, uncased, 300d vectors, 1.75 GB download)
3. Common Crawl (840B tokens, 2.2M vocab, cased, 300d vectors, 2.03 GB download)
4. Twitter (2B tweets, 27B tokens, 1.2M vocab, uncased, 25d, 50d, 100d, & 200d vectors, 1.42 GB download)

## GloVe Twitter

In [18]:
models = [glove_1, glove_2, glove_3, glove_4]

WEAT_results_1 = run_queries(
    WEAT,
    queries,
    models,
    metric_params={'preprocessor_args': {
        'lowercase': True
    }},
    aggregate_results=True,
    aggregation_function='abs_avg',
    queries_set_name='WEAT Queries')

%store WEAT_results_1
WEAT_results_1

Stored 'WEAT_results_1' (DataFrame)


,Flowers and Insects wrt Pleasant(5) and Unpleasant(5),Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9),Male names and Female names wrt Career and Family,Math and Arts wrt Male terms and Female terms,Science and Arts 2 wrt Male terms and Female terms,Mental disease and Physical disease wrt Temporary and Permanent,Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),WEAT: WEAT Queries average of abs values score
model_name,,,,,,,,,,,
glove twitter dim=25,3.983256,1.727524,3.752915,1.838223,1.539729,0.817561,0.042542,0.090977,0.267537,0.329010,1.438927
glove twitter dim=50,3.059114,2.611351,2.564336,1.432154,1.184290,0.736073,-0.211789,-0.180491,0.357402,0.354238,1.269124
glove twitter dim=100,1.998491,1.815693,2.188712,1.214762,1.380674,0.653838,-0.060213,0.004055,0.626559,0.360366,1.030336
glove twitter dim=200,1.446090,1.209590,1.674214,0.918201,1.160593,0.537437,-0.035495,0.063129,0.370728,0.224397,0.763987


In [19]:
WEAT_ranking_1 = create_ranking([WEAT_results_1])

%store WEAT_ranking_1
WEAT_ranking_1

Stored 'WEAT_ranking_1' (DataFrame)


,WEAT: WEAT Queries average of abs values score
model_name,
glove twitter dim=25,4.0
glove twitter dim=50,3.0
glove twitter dim=100,2.0
glove twitter dim=200,1.0


In [20]:
models = [glove_1, glove_2, glove_3, glove_4]

WEAT_results_1_2 = run_queries(
    WEAT,
    queries,
    models,
    aggregate_results=True,
    aggregation_function='abs_avg',
    queries_set_name='WEAT Queries')

%store WEAT_results_1_2
WEAT_results_1_2

ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold paramete

ERROR:root:At least one set of 'European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male names and Female names wrt Career and Family' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male names and Female names wrt Career and Family' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male names and Female names wrt Career and Family' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male names and Female names wrt Career and Family' 

Stored 'WEAT_results_1_2' (DataFrame)


,Flowers and Insects wrt Pleasant(5) and Unpleasant(5),Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9),Male names and Female names wrt Career and Family,Math and Arts wrt Male terms and Female terms,Science and Arts 2 wrt Male terms and Female terms,Mental disease and Physical disease wrt Temporary and Permanent,Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),WEAT: WEAT Queries average of abs values score
model_name,,,,,,,,,,,
glove twitter dim=25,3.983256,1.727524,NaN,NaN,NaN,NaN,0.042542,NaN,0.267537,NaN,1.505215
glove twitter dim=50,3.059114,2.611351,NaN,NaN,NaN,NaN,-0.211789,NaN,0.357402,NaN,1.559914
glove twitter dim=100,1.998491,1.815693,NaN,NaN,NaN,NaN,-0.060213,NaN,0.626559,NaN,1.125239
glove twitter dim=200,1.446090,1.209590,NaN,NaN,NaN,NaN,-0.035495,NaN,0.370728,NaN,0.765476


In [21]:
WEAT_ranking_1_2 = create_ranking([WEAT_results_1_2])

%store WEAT_ranking_1_2
WEAT_ranking_1_2

Stored 'WEAT_ranking_1_2' (DataFrame)


,WEAT: WEAT Queries average of abs values score
model_name,
glove twitter dim=25,3.0
glove twitter dim=50,4.0
glove twitter dim=100,2.0
glove twitter dim=200,1.0


## GloVe Wiki Gigaword

In [22]:
# Load the models

models = [glove_5, glove_6, glove_7, glove_8]

WEAT_results_2 = run_queries(
    WEAT,
    queries,
    models,
    metric_params={'preprocessor_args': {
        'lowercase': True
    }},
    aggregate_results=True,
    aggregation_function='abs_avg',
    queries_set_name='WEAT Queries')

%store WEAT_results_2
WEAT_results_2

Stored 'WEAT_results_2' (DataFrame)


,Flowers and Insects wrt Pleasant(5) and Unpleasant(5),Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9),Male names and Female names wrt Career and Family,Math and Arts wrt Male terms and Female terms,Science and Arts 2 wrt Male terms and Female terms,Mental disease and Physical disease wrt Temporary and Permanent,Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),WEAT: WEAT Queries average of abs values score
model_name,,,,,,,,,,,
glove wiki dim=50,2.060058,3.765126,1.893190,0.872163,1.330919,2.317143,0.468289,0.402531,0.585196,0.319580,1.401419
glove wiki dim=100,1.697702,2.079314,1.552911,0.970870,1.433715,1.732284,0.365643,0.252937,0.662625,0.335220,1.108322
glove wiki dim=200,1.172520,2.252081,1.442926,0.828176,1.114291,1.494439,0.274682,0.334962,0.688102,0.199697,0.980188
glove wiki dim=300,1.265150,1.905905,1.278590,0.847603,1.068681,1.319492,0.242773,0.318566,0.565714,0.212137,0.902461


In [23]:
WEAT_ranking_2 = create_ranking([WEAT_results_2])

%store WEAT_ranking_2
WEAT_ranking_2

Stored 'WEAT_ranking_2' (DataFrame)


,WEAT: WEAT Queries average of abs values score
model_name,
glove wiki dim=50,4.0
glove wiki dim=100,3.0
glove wiki dim=200,2.0
glove wiki dim=300,1.0


In [24]:
# Load the models

models = [glove_5, glove_6, glove_7, glove_8]

WEAT_results_2_2 = run_queries(
    WEAT,
    queries,
    models,
    metric_params={'preprocessor_args': {
        'lowercase': True
    }},
    aggregate_results=True,
    aggregation_function='abs_avg',
    queries_set_name='WEAT Queries')

%store WEAT_results_2_2
WEAT_results_2_2

Stored 'WEAT_results_2_2' (DataFrame)


,Flowers and Insects wrt Pleasant(5) and Unpleasant(5),Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9),Male names and Female names wrt Career and Family,Math and Arts wrt Male terms and Female terms,Science and Arts 2 wrt Male terms and Female terms,Mental disease and Physical disease wrt Temporary and Permanent,Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),WEAT: WEAT Queries average of abs values score
model_name,,,,,,,,,,,
glove wiki dim=50,2.060058,3.765126,1.893190,0.872163,1.330919,2.317143,0.468289,0.402531,0.585196,0.319580,1.401419
glove wiki dim=100,1.697702,2.079314,1.552911,0.970870,1.433715,1.732284,0.365643,0.252937,0.662625,0.335220,1.108322
glove wiki dim=200,1.172520,2.252081,1.442926,0.828176,1.114291,1.494439,0.274682,0.334962,0.688102,0.199697,0.980188
glove wiki dim=300,1.265150,1.905905,1.278590,0.847603,1.068681,1.319492,0.242773,0.318566,0.565714,0.212137,0.902461


In [25]:
WEAT_ranking_2_2 = create_ranking([WEAT_results_2_2])

%store WEAT_ranking_2_2
WEAT_ranking_2_2

Stored 'WEAT_ranking_2_2' (DataFrame)


,WEAT: WEAT Queries average of abs values score
model_name,
glove wiki dim=50,4.0
glove wiki dim=100,3.0
glove wiki dim=200,2.0
glove wiki dim=300,1.0


## GloVe Common Crawl - lowercase:True

In [11]:
#load('/Users/macbookair/Desktop/postgraduate/dissertation/glove.42B.300d.txt')
#load('/Users/macbookair/Desktop/postgraduate/dissertation/glove.840B.300d.txt')

models = [glove_9, glove_10]

WEAT_results_3 = run_queries(
    WEAT,
    queries,
    models,
    metric_params={'preprocessor_args': {
        'lowercase': True
    }},
    aggregate_results=True,
    aggregation_function='abs_avg',
    queries_set_name='WEAT Queries')

%store WEAT_results_3
WEAT_results_3

Stored 'WEAT_results_3' (DataFrame)


,Flowers and Insects wrt Pleasant(5) and Unpleasant(5),Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9),Male names and Female names wrt Career and Family,Math and Arts wrt Male terms and Female terms,Science and Arts 2 wrt Male terms and Female terms,Mental disease and Physical disease wrt Temporary and Permanent,Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),WEAT: WEAT Queries average of abs values score
model_name,,,,,,,,,,,
glove common crawl 42B dim=300,2.327526,1.840402,1.828122,0.893584,0.949063,0.738134,0.259692,0.234783,0.587961,0.212755,0.987202
glove common crawl 840B dim=300,2.238165,2.290555,1.520552,0.615430,0.860291,0.651596,0.198923,0.299863,0.576485,0.316962,0.956882


In [12]:
WEAT_ranking_3 = create_ranking([WEAT_results_3])

%store WEAT_ranking_3
WEAT_ranking_3

Stored 'WEAT_ranking_3' (DataFrame)


,WEAT: WEAT Queries average of abs values score
model_name,
glove common crawl 42B dim=300,2.0
glove common crawl 840B dim=300,1.0


## GloVe Common Crawl - lowercase:False

In [28]:
models = [glove_9, glove_10]

WEAT_results_4 = run_queries(
    WEAT,
    queries,
    models,
    aggregate_results=True,
    aggregation_function='abs_avg',
    queries_set_name='WEAT Queries')

%store WEAT_results_4
WEAT_results_4

ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male names and Female names wrt Career and Family' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
E

Stored 'WEAT_results_4' (DataFrame)


,Flowers and Insects wrt Pleasant(5) and Unpleasant(5),Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9),Male names and Female names wrt Career and Family,Math and Arts wrt Male terms and Female terms,Science and Arts 2 wrt Male terms and Female terms,Mental disease and Physical disease wrt Temporary and Permanent,Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),WEAT: WEAT Queries average of abs values score
model_name,,,,,,,,,,,
glove common crawl 42B dim=300,2.327525,1.840402,NaN,NaN,NaN,NaN,0.259692,NaN,0.587961,NaN,1.253895
glove common crawl 840B dim=300,2.238165,2.290555,1.863447,0.971278,1.112238,1.26698,0.198923,0.313517,0.576485,0.354009,1.118560


In [10]:
WEAT_ranking_4 = create_ranking([WEAT_results_4])

%store WEAT_ranking_4
WEAT_ranking_4

Stored 'WEAT_ranking_4' (DataFrame)


,WEAT: WEAT Queries average of abs values score
model_name,
glove common crawl 42B dim=300,2.0
glove common crawl 840B dim=300,1.0


# word2vec
Pre-trained vectors trained on part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases.

In [5]:
models = [model_w2v]

WEAT_results_5 = run_queries(
    WEAT,
    queries,
    models,
    metric_params={'preprocessor_args': {
        'lowercase': True
    }},
    aggregate_results=True,
    aggregation_function='abs_avg',
    queries_set_name='WEAT Queries')

%store WEAT_results_5
WEAT_results_5

ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This q

Stored 'WEAT_results_5' (DataFrame)


,Flowers and Insects wrt Pleasant(5) and Unpleasant(5),Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9),Male names and Female names wrt Career and Family,Math and Arts wrt Male terms and Female terms,Science and Arts 2 wrt Male terms and Female terms,Mental disease and Physical disease wrt Temporary and Permanent,Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),WEAT: WEAT Queries average of abs values score
model_name,,,,,,,,,,,
word2vec google dim=300,1.407829,1.747649,NaN,NaN,NaN,0.625607,0.225461,0.250128,0.338592,NaN,0.765878


There is a whole column that has no results. As the warnings point out, when transforming the words of the sets into embeddings, there is a loss of words that is greater than the allowed by the parameter lost_vocabulary_threshold. Therefore, all those queries return np.nan. In this case, it would be very useful to use the word preprocessors seen above.

If more than 20% (by default) of the words from any of the word sets of the query are not included in the word embedding model, the metric will return Nan. This behavior can be changed using a float number parameter called lost_vocabulary_threshold. 

More than 20% of words from 'African american names(7)', 'African american names(5)' and 'Old peoples names' are not included in the model

In [23]:
models = [model_w2v]

WEAT_results_5_2 = run_queries(
    WEAT,
    queries,
    models,
    metric_params={'preprocessor_args': {
        'lowercase': True
    }},
    lost_vocabulary_threshold= 0.1,
    aggregate_results=True,
    aggregation_function='abs_avg',
    queries_set_name='WEAT Queries')

%store WEAT_results_5_2
WEAT_results_5_2

Stored 'WEAT_results_5_2' (DataFrame)


,Flowers and Insects wrt Pleasant(5) and Unpleasant(5),Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9),Male names and Female names wrt Career and Family,Math and Arts wrt Male terms and Female terms,Science and Arts 2 wrt Male terms and Female terms,Mental disease and Physical disease wrt Temporary and Permanent,Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),WEAT: WEAT Queries average of abs values score
model_name,,,,,,,,,,,
word2vec google dim=300,1.407829,1.747649,0.453614,0.453276,0.33806,1.25161,0.225461,0.293065,0.338592,-0.048873,0.655803


In [23]:
models = [model_w2v]

WEAT_results_5_3 = run_queries(
    WEAT,
    queries,
    models,
    aggregate_results=True,
    aggregation_function='abs_avg',
    queries_set_name='WEAT Queries')

%store WEAT_results_5_3
WEAT_results_5_3

Stored 'WEAT_results_5_2' (DataFrame)


,Flowers and Insects wrt Pleasant(5) and Unpleasant(5),Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9),Male names and Female names wrt Career and Family,Math and Arts wrt Male terms and Female terms,Science and Arts 2 wrt Male terms and Female terms,Mental disease and Physical disease wrt Temporary and Permanent,Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),WEAT: WEAT Queries average of abs values score
model_name,,,,,,,,,,,
word2vec google dim=300,1.407829,1.747649,0.453614,0.453276,0.33806,1.25161,0.225461,0.293065,0.338592,-0.048873,0.655803


# fastText crawl
1. fasttext crawl dim=300: 2 million word vectors trained on Common Crawl (600B tokens).
2. fasttext crawl subwords dim=300: 2 million word vectors trained with subword information on Common Crawl (600B tokens).

A subword is the internal structure of words. Classic word representation cannot handle unseen word or rare word well. Character embeddings is one of the solution to overcome out-of-vocabulary (OOV). However, it may too fine-grained any missing some important information. Subword is in between word and character. It is not too fine-grained while able to handle unseen word and rare word. For example, we can split “subword” to “sub” and “word”. In other word we use two vector (i.e. “sub” and “word”) to represent “subword”.

More than 20% of words from 'African american names(7)' are not included in setting 'fasttext crawl subwords dim=300'

In [19]:
models = [fasttext_3, fasttext_4]

WEAT_results_6 = run_queries(
    WEAT,
    queries,
    models,
    metric_params={'preprocessor_args': {
        'lowercase': True
    }},
    aggregate_results=True,
    aggregation_function='abs_avg',
    queries_set_name='WEAT Queries')

%store WEAT_results_6
WEAT_results_6

ERROR:root:At least one set of 'European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.


Stored 'WEAT_results_6' (DataFrame)


,Flowers and Insects wrt Pleasant(5) and Unpleasant(5),Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9),Male names and Female names wrt Career and Family,Math and Arts wrt Male terms and Female terms,Science and Arts 2 wrt Male terms and Female terms,Mental disease and Physical disease wrt Temporary and Permanent,Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),WEAT: WEAT Queries average of abs values score
model_name,,,,,,,,,,,
fasttext crawl dim=300,1.678853,1.719299,0.606784,0.159027,0.349127,0.453902,0.220737,0.312242,0.208390,0.211301,0.591966
fasttext crawl subwords dim=300,1.668807,1.177889,0.977537,NaN,NaN,0.393544,0.267563,0.200230,0.285504,0.140782,0.638982


In [20]:
WEAT_ranking_6 = create_ranking([WEAT_results_6])

%store WEAT_ranking_6
WEAT_ranking_6

Stored 'WEAT_ranking_6' (DataFrame)


,WEAT: WEAT Queries average of abs values score
model_name,
fasttext crawl dim=300,1.0
fasttext crawl subwords dim=300,2.0


## Remove Preprocessing

In [26]:
models = [fasttext_3, fasttext_4]

WEAT_results_6_2 = run_queries(
    WEAT,
    queries,
    models,
    aggregate_results=True,
    aggregation_function='abs_avg',
    queries_set_name='WEAT Queries')

%store WEAT_results_6_2
WEAT_results_6_2

Stored 'WEAT_results_6_2' (DataFrame)


,Flowers and Insects wrt Pleasant(5) and Unpleasant(5),Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9),Male names and Female names wrt Career and Family,Math and Arts wrt Male terms and Female terms,Science and Arts 2 wrt Male terms and Female terms,Mental disease and Physical disease wrt Temporary and Permanent,Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),WEAT: WEAT Queries average of abs values score
model_name,,,,,,,,,,,
fasttext crawl dim=300,1.678853,1.719299,0.639385,0.327617,0.544679,0.504557,0.220737,0.300546,0.208390,0.325519,0.646958
fasttext crawl subwords dim=300,1.668807,1.177889,0.901791,0.387363,0.552362,0.431821,0.267563,0.169132,0.285504,0.213893,0.605613


More than 20% of words from 'African american names(7)' are not included in setting 'fasttext crawl subwords dim=300'

In [27]:
WEAT_ranking_6_2 = create_ranking([WEAT_results_6_2])

%store WEAT_ranking_6_2
WEAT_ranking_6_2

Stored 'WEAT_ranking_6_2' (DataFrame)


,WEAT: WEAT Queries average of abs values score
model_name,
fasttext crawl dim=300,2.0
fasttext crawl subwords dim=300,1.0


# fastText wiki news
1. fasttext wiki dim=300: 1 million word vectors trained on Wikipedia 2017, UMBC webbase corpus and statmt.org news dataset (16B tokens).
2. fasttext wiki subwords dim=300: 1 million word vectors trained with subword infomation on Wikipedia 2017, UMBC webbase corpus and statmt.org news dataset (16B tokens).

In [15]:
models = [fasttext_1, fasttext_2]

WEAT_results_7 = run_queries(
    WEAT,
    queries,
    models,
    metric_params={'preprocessor_args': {
        'lowercase': True
    }},
    aggregate_results=True,
    aggregation_function='abs_avg',
    queries_set_name='WEAT Queries')

%store WEAT_results_7
WEAT_results_7

ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold paramete

Stored 'WEAT_results_7' (DataFrame)


,Flowers and Insects wrt Pleasant(5) and Unpleasant(5),Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9),Male names and Female names wrt Career and Family,Math and Arts wrt Male terms and Female terms,Science and Arts 2 wrt Male terms and Female terms,Mental disease and Physical disease wrt Temporary and Permanent,Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),WEAT: WEAT Queries average of abs values score
model_name,,,,,,,,,,,
fasttext wiki subwords dim=300,1.358939,1.649030,NaN,NaN,NaN,0.439045,0.198445,0.102133,0.195474,NaN,0.657178
fasttext wiki dim=300,0.777454,1.347168,NaN,NaN,NaN,0.511297,0.202839,0.137419,0.243172,NaN,0.536558


More than 20% of words from 'African american names(7)' and 'Old peoples names' are not included in both settings

In [17]:
WEAT_ranking_7 = create_ranking([WEAT_results_7])

%store WEAT_ranking_7
WEAT_ranking_7

Stored 'WEAT_ranking_7' (DataFrame)


,WEAT: WEAT Queries average of abs values score
model_name,
fasttext wiki subwords dim=300,2.0
fasttext wiki dim=300,1.0


In [26]:
models = [fasttext_1, fasttext_2]

WEAT_results_7_2 = run_queries(
    WEAT,
    queries,
    models,
    metric_params={'preprocessor_args': {
        'lowercase': True
    }},
    aggregate_results=True,
    aggregation_function='abs_avg',
    queries_set_name='WEAT Queries')

%store WEAT_results_7_2
WEAT_results_7_2

ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold paramete

Stored 'WEAT_results_7_2' (DataFrame)


,Flowers and Insects wrt Pleasant(5) and Unpleasant(5),Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(5) and Unpleasant(5),European american names(7) and African american names(7) wrt Pleasant(9) and Unpleasant(9),Male names and Female names wrt Career and Family,Math and Arts wrt Male terms and Female terms,Science and Arts 2 wrt Male terms and Female terms,Mental disease and Physical disease wrt Temporary and Permanent,Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),WEAT: WEAT Queries average of abs values score
model_name,,,,,,,,,,,
fasttext wiki subwords dim=300,1.358939,1.649030,NaN,NaN,NaN,0.439045,0.198445,0.102134,0.195474,NaN,0.657178
fasttext wiki dim=300,0.777454,1.347168,NaN,NaN,NaN,0.511297,0.202839,0.137419,0.243173,NaN,0.536558


In [27]:
WEAT_ranking_7_2 = create_ranking([WEAT_results_7_2])

%store WEAT_ranking_7_2
WEAT_ranking_7_2

Stored 'WEAT_ranking_7_2' (DataFrame)


,WEAT: WEAT Queries average of abs values score
model_name,
fasttext wiki subwords dim=300,2.0
fasttext wiki dim=300,1.0
